IMPORT


In [28]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import math


from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.api import OLS
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = 2000
pd.options.display.max_rows = 2000


#path = '/Koding_With_Kolesh/challenges/RMB_NOWCAST_APRIL/'
cpi = pd.read_csv('CPI_Historic_Values_Zindi_May_23.csv')
#vehicles = pd.read_csv('Naamsa_Vehicle_Sales.csv')
seed = 3

PIVOTING THE DATA


In [29]:
cpi_pivot = cpi.pivot(index = 'Month', columns = 'Category', values = 'Value').reset_index()#changing from a long format to a wide format,
cpi_pivot['Month'] = pd.to_datetime(cpi_pivot['Month'])
cpi_pivot = cpi_pivot.sort_values("Month").reset_index(drop=True)

ADDING JUNE and july'S DATA MANUALLY FROM https://www.statssa.gov.za/publications/P0141/P0141June2023.pdf and https://www.statssa.gov.za/publications/P0141/P0141July2023.pdf


In [30]:
date_str = '2023-06-30'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_june = [110.9, 104.3, 99.6, 110.4, 118.3, 109.8, 110.8, 107.7, 105.4, 109.6, 105.3, 110.0, 112.3]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[17, col] = cpi_of_june[i]
date_str = '2023-07-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)
cpi_of_july = [111.5, 104.5, 99.5, 110.4, 118.5, 110.8, 110.6, 108.2 , 108.4, 109.9, 105.5, 110, 112.6]

for i, col in enumerate(cpi_pivot.columns[1:]):
    cpi_pivot.at[18, col] = cpi_of_july[i]
cpi_pivot

,Month,Alcoholic beverages and tobacco,Clothing and footwear,Communication,Education,Food and non-alcoholic beverages,Headline_CPI,Health,Household contents and services,Housing and utilities,Miscellaneous goods and services,Recreation and culture,Restaurants and hotels,Transport
0,2022-01-31,100.0,100.2,99.8,100.0,100.9,100.2,100.1,100.4,100.0,100.6,100.2,101.2,98.9
1,2022-02-28,100.6,100.3,99.7,100.0,101.8,100.8,103.2,100.4,100.1,101.5,100.3,101.9,100.6
2,2022-03-31,102.8,100.5,99.7,104.4,102.4,101.8,103.7,100.8,100.5,101.9,100.4,103.0,103.7
3,2022-04-30,103.6,100.7,99.6,104.4,103.1,102.4,104.0,101.1,100.5,102.8,100.7,103.3,105.1
4,2022-05-31,104.4,100.9,99.9,104.4,105.3,103.1,104.3,101.8,100.6,102.9,101.0,103.8,105.9
5,2022-06-30,104.5,101.3,100.3,104.4,106.6,104.2,104.6,103.0,101.2,103.0,101.2,104.2,110.3
6,2022-07-31,104.4,101.6,99.4,104.4,107.8,105.8,104.1,103.3,103.1,103.4,102.3,104.6,115.6
7,2022-08-31,104.7,102.1,100.1,104.4,109.7,106.0,104.2,103.7,103.2,103.5,102.4,104.3,114.5
8,2022-09-30,105.6,102.5,99.9,104.4,110.3,106.1,104.0,104.5,103.7,103.8,102.5,106.2,112.0
9,2022-10-31,106.1,102.6,99.8,104.4,111.3,106.5,104.5,105.0,103.8,104.7,102.7,107.4,111.7


ADD THE August ROW


In [31]:
date_str = '2023-08-31'
date_obj = pd.to_datetime(date_str)
new_row = pd.DataFrame({'Month': [date_obj]})
cpi_pivot = pd.concat([cpi_pivot, new_row]).reset_index(drop=True)

FEATURE ENGINNERING


In [32]:
feats_to_lag = cpi_pivot.columns[1:].to_list()
for col in feats_to_lag:
    for i in range(1,5):
        cpi_pivot[f'prev_{i}_month_{col}'] = cpi_pivot[col].shift(i)

HANDLE MISSING DATA


In [33]:
cpi_pivot = cpi_pivot.drop(0)
cpi_pivot = cpi_pivot.bfill()

TRAIN AND VALIDATION


In [34]:
train = cpi_pivot[cpi_pivot['Month'] != "2023-08-31"]
test = cpi_pivot[cpi_pivot['Month'] == "2023-08-31"]

training_set = train[train['Month']!= '2023-07-31']
validation_set = train[train['Month']== '2023-07-31']

train.shape, test.shape, training_set.shape, validation_set.shape

((18, 66), (1, 66), (17, 66), (1, 66))

MODELING


In [35]:
import optuna


target_cols = ['Alcoholic beverages and tobacco', 'Clothing and footwear',
       'Communication', 'Education', 'Food and non-alcoholic beverages',
       'Headline_CPI', 'Health', 'Household contents and services',
       'Housing and utilities', 'Miscellaneous goods and services',
       'Recreation and culture', 'Restaurants and hotels ', 'Transport']

#if you add additional data sources that have no value in the predicting month , drop it, now that you have their lags
features= [col for col in train.columns if col not in target_cols + ['Month']]


X_train = training_set[features]
y_train = training_set[target_cols]

X_val = validation_set[features]
y_val = validation_set[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}
bp = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

def objective(trial):
    alpha = trial.suggest_loguniform('alpha', 1e-5, 100)  # Ridge regularization strength
    fit_intercept = trial.suggest_categorical('fit_intercept', [True, False])
    copy_X = trial.suggest_categorical('copy_X', [True, False])
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    solver = trial.suggest_categorical('solver', ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'])

    r_model = Ridge(alpha=alpha, fit_intercept=fit_intercept, copy_X=copy_X, max_iter=max_iter, tol=tol, solver=solver,)
    r_model.fit(X_train, y_train[target_col])
    y_pred_colr = r_model.predict(X_val)
    return mean_squared_error(y_val[target_col], y_pred_colr)

#training
for target_col in target_cols:
    #study = optuna.create_study(direction='minimize')
    #study.optimize(objective, n_trials=50)
    #best_params = study.best_params
    
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.067, random_state=seed)
    el_model = ElasticNet(alpha=0.1, l1_ratio=0.5, warm_start=True, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    #bp[target_col] = best_params
#validation
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)

#scoring

y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

df = pd.DataFrame({'y_pred': y_pred.flatten(), 'y_val': y_val.values.flatten()})
dfx = pd.DataFrame({'y_pred': y_predx.flatten(), 'y_val': y_val.values.flatten()})
dfl = pd.DataFrame({'y_pred': y_predl.flatten(), 'y_val': y_val.values.flatten()})
dfr = pd.DataFrame({'y_pred': y_predr.flatten(), 'y_val': y_val.values.flatten()})
dfel = pd.DataFrame({'y_pred': y_predel.flatten(), 'y_val': y_val.values.flatten()})

#calculate the rmse
rmse = np.sqrt(mean_squared_error(df['y_pred'], df['y_val']))
rmsex = np.sqrt(mean_squared_error(dfx['y_pred'], df['y_val']))
rmsel = np.sqrt(mean_squared_error(dfl['y_pred'], df['y_val']))
rmser = np.sqrt(mean_squared_error(dfr['y_pred'], df['y_val']))
rmseel = np.sqrt(mean_squared_error(dfel['y_pred'], df['y_val']))

print(f'RMSE of Liner Regression: {rmse}') 
print(f'RMSE of XGB: {rmsex}') 
print(f'RMSE of Lasso Regression: {rmsel}') 
print(f'RMSE of Riged Regression: {rmser}') 
print(f'RMSE of Elastic Regression: {rmseel}') 

RMSE of Liner Regression: 1.2179142897989084
RMSE of XGB: 0.9239730000576405
RMSE of Lasso Regression: 1.0660095553164495
RMSE of Riged Regression: 1.3764569824235542
RMSE of Elastic Regression: 1.7466824496596614


In [36]:
X_train = train[features]
y_train = train[target_cols]

X_val = test[features]
y_val = test[target_cols]

x_models = {}
lr_models = {}
l_models = {}
r_models = {}
el_models = {}

y_pred = []
y_predx = []
y_predl = []
y_predr = []
y_predel = []

scaler = MinMaxScaler()

#training
for target_col in target_cols:
    lr_model = LinearRegression()
    x_model = XGBRegressor(seed=seed)
    r_model = Ridge(random_state=seed)
    l_model = Lasso(alpha=0.067, random_state=seed)
    el_model = ElasticNet(alpha=0.1, l1_ratio=0.5, warm_start=True, random_state= seed)
    
    X_train_scaled = scaler.fit_transform(X_train)
    
    lr_model.fit(X_train_scaled, y_train[target_col])
    x_model.fit(X_train_scaled, y_train[target_col])
    r_model.fit(X_train, y_train[target_col])
    l_model.fit(X_train_scaled, y_train[target_col])
    el_model.fit(X_train_scaled, y_train[target_col])
    
    lr_models[target_col] = lr_model
    l_models[target_col] = l_model
    r_models[target_col] = r_model
    el_models[target_col] = el_model
    x_models[target_col] = x_model
    
#pridiction
for target_col in target_cols:
    lr_model = lr_models[target_col]
    x_model = x_models[target_col]
    l_model = l_models[target_col]
    r_model = r_models[target_col]
    el_model = el_models[target_col]
    
    X_val_scaled = scaler.transform(X_val)
    
    y_pred_col = lr_model.predict(X_val_scaled)
    y_pred_colx = x_model.predict(X_val_scaled)
    y_pred_coll = l_model.predict(X_val_scaled)
    y_pred_colr = r_model.predict(X_val)
    y_pred_colel = el_model.predict(X_val_scaled)
    
    y_pred.append(y_pred_col)
    y_predx.append(y_pred_colx)
    y_predl.append(y_pred_coll)
    y_predr.append(y_pred_colr)
    y_predel.append(y_pred_colel)
    
y_pred = np.array(y_pred).T
y_predx = np.array(y_predx).T
y_predl = np.array(y_predl).T
y_predr = np.array(y_predr).T
y_predel = np.array(y_predel).T

print(f'pridiction of Liner Regression: {y_pred}') 
print(f'prediction of XGB: {y_predx}') 
print(f'pridiction of Riged Regression: {y_predr}') 
print(f'prediction of Lasso Regression: {y_predl}') 
print(f'pridiction of Elastic Net Regression: {y_predel}') 

pridiction of Liner Regression: [[111.31066459 105.06083029  99.58975554 109.93043024 118.71612253
  111.41335506 112.39309642 108.39284407 108.64567806 110.93284663
  105.31366846 111.96022249 114.44853858]]
prediction of XGB: [[111.49858  104.498886  99.49246  110.39958  118.49875  110.798965
  110.798805 108.1308   108.39877  109.898705 105.49876  109.997154
  112.90594 ]]
pridiction of Riged Regression: [[111.22425135 104.77297648  99.50625806 110.16266525 118.57036986
  111.03851768 111.61401636 108.0085744  108.36554536 110.40899561
  105.48162979 110.74304001 113.78994901]]
prediction of Lasso Regression: [[111.3545613  104.06807803  99.67777778 110.53432691 119.40337471
  110.62890716 110.37696477 108.2645868  107.5755366  109.77639872
  105.13011733 110.52934673 112.53042585]]
pridiction of Elastic Net Regression: [[111.24209917 104.40964989  99.67777778 109.94798507 119.86343553
  110.8870156  110.32610656 108.54974845 107.1121003  109.82694543
  105.37923235 110.48314817 114

In [37]:
def prepSub(y_pred:list, target_cols: list, test, prefix:str):
    sub_df = pd.DataFrame(y_pred, columns=target_cols)
    sub_df['Month'] = test['Month']
    
    sub_df.set_index('Month', inplace=True)
    sub_df.columns = [prefix+'_' + col.lower().replace('_', ' ').strip() for col in sub_df.columns]
    sub_df.rename(columns= {f"{prefix}_headline cpi": f"{prefix}_headline CPI"}, inplace=True)
    
    sub_df = pd.melt(sub_df.reset_index(), id_vars= ['Month'], var_name= 'ID', value_name= 'Value')
    
    return sub_df[['ID', 'Value']]

In [38]:
sub = prepSub(y_predl, target_cols, test, 'August') 
sub.to_csv('out/lassoa0067_with_lag5_historicj.csv', index=False)